# Instructions
The following code was designed in order to load in and score motion/freezing data for a single video file. It allows for cropping of the video frame to reduce the influence of miniscope/optogenetic cables.  After initally loading in the video, the user is able to crop the video using a box selection tool.  Motion is then detected by capturing the number of pixels whose frame by frame grayscale change value exceeds a threshold (determined using FreezeAnalysis_Calibration.ipynb).  Freezing is then assessed by calculating when motion drops below a user-defined threshold.  The user is able to visualize raw video, animal motion, and whether an animal is deemed to be freezing in order optimize parameters.  Final output is a csv which provides frame by frame motion and freezing across the session.  Additionally, a summary file can be generated which allows the user to get motion/freezing for specified time bins.  With the exception of setting the file path and small number of parameters (ie frames per second in video, freezing threshold), the user can run through code.  Once parameters are found that work well, batch processing of multiple videos can be performed using FreezeAnalysis_BatchProcess.ipynb.

---
# 1. Load Necessary Packages
The following code loads neccessary packages and need not be changed by the user.

In [145]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import holoviews as hv
import FreezeAnalysis_Functions as fz

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
# 2. User Sets Directory and File Information
Below the user sets the directory file path (path of the folder where the video file is), the file name of the video to be analyzed (including the extension), the frame on which the analysis is to begin (0 is the first frame), and the last frame to be analyzed (set `'end' : None` if processing entire video). Additionally, by modifying 'stretch', one can alter the relative width/height of the presented output.

***Windows Users:*** Place an 'r' in front directory path (e.g. r"zp\Videos") to avoid mishandling of forward slashes.

In [396]:
video_dict = {
    'dpath' : "/users/adrianamendez/desktop/videos_auto/",  
    'file' : 'dlx-vHip-BLA-1.avi',
    'start' : 0, 
    'end' : None 
}
stretch = dict(width=1, height=1)

---
# 3. Load Video and Crop Frame if Desired
To crop video frame, after running code below, select box selection tool below image (square with a plus sign).  To start drawing region to be included in analyis, double click image.  Double click again to finalize region.  If you decide to change region, it is best to rerun this cell and subsequent steps.

If the size of the image is too small/large, alter the first line of code.  100 is the standard size.  200 will produce an image 2x the size, and so on.

In [397]:
%%output size=100

image,crop,video_dict=fz.LoadAndCrop(video_dict,stretch,cropmethod="Box")
image

file: /users/adrianamendez/desktop/videos_auto/dlx-vHip-BLA-1.avi
total frames: 61192


:Overlay
   .Image.I    :Image   [x,y]   (z)
   .Polygons.I :Polygons   [x,y]

---
# 4. Analyze Motion Across Session

### 4a. User Sets Parameters for Motion Analysis
Below, user setes grayscale difference value required for pixel to be counted as changing

In [399]:
mt_cutoff = 15

### 4b. Detect Motion and Plot
Here, code loops through all frames and detects number of pixels whose grayscale change exceeds cutoff per frame.  Motion vector is populated with these values.  In addition to interactive plot options via toolbar, one can also change the plot size by changing h and w in code below (e.g `h,w = 300,2000`)

In [400]:
Motion = fz.Measure_Motion(video_dict,mt_cutoff,crop=crop,SIGMA=1)  

h,w = 300,1000
motion_plot = hv.Curve((np.arange(len(Motion)),Motion),'Frame','Pixel Change').opts(
    height=h,width=w,line_width=1,color="steelblue",title="Motion Across Session")
motion_plot


:Curve   [Frame]   (Pixel Change)

---
# 5. Analyze Session Freezing

### 5a. User Selects Freezing Parameters
Below user sets 'FreezeThresh', the upper bound for freezing in frame-by-frame pixels changed, and 'MinDuration', the number of frames motion must be below FreezeThresh to begin accruing freezing.

In [401]:
FreezeThresh = 50
MinDuration = 5

### 5b. Measure Freezing and Save
Freezing is calculated with cell below.  In addition to interactive plot options with adjacent toolbar, one can also change the plot size by changing h and w in code below (e.g `h,w = 300,2000`)

In [403]:
%%output size=100

Freezing = fz.Measure_Freezing(Motion,FreezeThresh,MinDuration)  
fz.SaveData(video_dict,Motion,Freezing,mt_cutoff,FreezeThresh,MinDuration)
print('Average Freezing: {x}%'.format(x=np.average(Freezing)))

h,w = 300,1000 
freezing_plot = hv.Area(Freezing*(Motion.max()/100),'Frame','Motion').opts(
    color='lightgray',line_width=0,line_alpha=0)
motion_plot = hv.Curve((np.arange(len(Motion)),Motion),'Frame','Motion').opts(
    height=h,width=w,line_width=1, 
    color='steelblue',title="Motion Across Session with Freezing Highlighted in Gray")
freezing_plot*motion_plot

Average Freezing: 61.15705180585063%


:Overlay
   .Area.I  :Area   [Frame]   (Motion)
   .Curve.I :Curve   [Frame]   (Motion)

---
# 6. (Optional) Display Section of Video with Motion and Freezing
***After analyzing freezing*** a section of the video can be replayed and the animal's state - "Active/Freezing" - will be displayed.  User defines the start and end frames of the section they would like to watch.  Of note, if the user sets the start frame for location tracking to something other than 0 in Step 2, the frame start and end here will be relative to that. If the displayed size of the video should be changed, the user can set the `resize` key below to a tuple with the desired width and height, in pixel units... for example: `'resize' : (width,height)`.  The specified width and height should be integers. Lastly, the video can be saved by setting `'save_video' : True`. `fps` controls speed at which frames are presented.

In [386]:
display_dict = {
    'start' : 0, 
    'end' : 10,
    'fps' : 60,
    'resize' : None,
    'save_video' : False
}

fz.PlayVideo(video_dict,display_dict,Freezing,mt_cutoff,crop=crop,SIGMA=1)


IndexError: index 36396 is out of bounds for axis 0 with size 30595

---
# 7. (Optional) Create Binned Summary Report and Save
The code below allows the user to either save a csv containing summary data for user-defined bins (e.g. % time freezing for each minute) or a session-wide average. 

***If you only want a session avg***, set `bin_dict = None`

***If you are not using ROIs***, in the code below, set value of 'region_names' within function to None: `region_names=None`.  Otherwise, keep `region_names=region_names`

To specify bins, set bin_dict using the following notation, where start and stop represent time ***in frames***:

```
bin_dict = {
    'BinName1': (start, stop),
    'BinName2': (start, stop),
    'BinName3': (start, stop),
}
```

In [385]:
bin_dict = {
    
    #baseline 
    5: (0,300),
    10: (300,600),
    15: (600,900), 
    20: (900,1200), 
    25: (1200,1500),
    30: (1500, 1800), 
    35: (1800, 2100), 
    40: (2100, 2400), 
    45: (2400, 2700), 
    50: (2700, 3000), 
    55: (3000, 3300), 
    60: (3300, 3600), 
    65: (3600, 3900), 
    70: (3900, 4200), 
    75: (4200, 4500), 
    80: (4500, 4800), 
    85: (4800, 5100), 
    90: (5100, 5400), 
    95: (5400, 5700), 
    100: (5700, 6000),
    105: (6000, 6300), 
    110: (6300, 6600), 
    115: (6600, 6900), 
    120: (6900, 7200), 
    125: (7200, 7500), 
    130: (7500, 7800), 
    135: (7800, 8100), 
    140: (8100, 8400), 
    145: (8400, 8700), 
    150: (8700, 9000),
    155: (9000, 9300), 
    160: (9300, 9600), 
    165: (9600, 9900), 
    170: (9900, 10200), 
    175: (10200, 10500), 
    180: (10500, 10800),
    
    #first shock - without first 5 seconds
    
    1: (10200, 10500), 
    2: (10500, 10800), 
    3: (10800, 11100), 
    4: (11100, 11400), 
    
    #second shock - without first 5 seconds 
    
    6: (18545, 18845), 
    7: (18845, 19145), 
    8: (19145, 19445),
    9: (19445, 19745), 
    
    # third shock - without first 5 seconds 
    11: (27070, 27370), 
    12: (27370, 27670), 
    13: (27670, 27970), 
    14: (27970, 28270),
    
    #fourth shock - without first 5 seconds 
    
    16: ()
     
}

summary = fz.Summarize(video_dict,Motion,Freezing,FreezeThresh,MinDuration,mt_cutoff,bin_dict=bin_dict)
summary.to_csv(os.path.splitext(video_dict['fpath'])[0] + '_SummaryStats.csv')
summary

,File,FileLength,MotionCutoff,FreezeThresh,MinFreezeDuration,bin,range(f),Motion,Freezing
0,dlx-vHip-BLA-1.avi,30595.0,15.0,50.0,5.0,5,"(0, 300)",128.543333,12.333333
1,dlx-vHip-BLA-1.avi,30595.0,15.0,50.0,5.0,10,"(300, 600)",224.713333,25.666667
2,dlx-vHip-BLA-1.avi,30595.0,15.0,50.0,5.0,15,"(600, 900)",173.523333,9.666667
3,dlx-vHip-BLA-1.avi,30595.0,15.0,50.0,5.0,20,"(900, 1200)",153.920000,18.666667
4,dlx-vHip-BLA-1.avi,30595.0,15.0,50.0,5.0,25,"(1200, 1500)",136.616667,25.333333
5,dlx-vHip-BLA-1.avi,30595.0,15.0,50.0,5.0,30,"(1500, 1800)",184.520000,18.333333
6,dlx-vHip-BLA-1.avi,30595.0,15.0,50.0,5.0,35,"(1800, 2100)",195.506667,25.000000
7,dlx-vHip-BLA-1.avi,30595.0,15.0,50.0,5.0,40,"(2100, 2400)",108.543333,32.000000
8,dlx-vHip-BLA-1.avi,30595.0,15.0,50.0,5.0,45,"(2400, 2700)",225.536667,15.666667
9,dlx-vHip-BLA-1.avi,30595.0,15.0,50.0,5.0,50,"(2700, 3000)",198.266667,11.666667
